In [1]:
import tkinter as tk
from tkinter import Canvas
from PIL import Image, ImageTk
import threading
import speech_recognition as sr
import pyttsx3
import openai
import webbrowser
import subprocess

recognizer = sr.Recognizer()
engine = pyttsx3.init()

openai.api_key = 'sk-proj-fjG8wj9vYuaIjxyBFDF5T3BlbkFJFYKdF659ea7rHaUKnyxH'

# Main window
root = tk.Tk()
root.title("Shakti - Voice Assistant")
root.attributes('-fullscreen', True)  # Open in fullscreen mode
root.configure(bg='black')

# Exit fullscreen with ESC key
root.bind("<Escape>", lambda event: root.attributes('-fullscreen', False))

try:
    bg_image_path = "sci-fi background.jpg" 
    bg_image = Image.open(bg_image_path)
    bg_image = bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
    bg_photo = ImageTk.PhotoImage(bg_image)
    
    # Keep a reference to the image to prevent garbage collection
    root.bg_photo = bg_photo

    bg_label = tk.Label(root, image=bg_photo)
    bg_label.image = bg_photo  # Store a reference to the image
    bg_label.place(relwidth=1, relheight=1)
except Exception as e:
    print(f"Error loading background image: {e}")

try:
    # Logo and creator's name
    logo_img_path = "RoboAI logo.png"  # Replace with your logo image path
    logo_img = Image.open(logo_img_path)
    logo_img = logo_img.resize((100, 100), Image.LANCZOS)
    logo_photo = ImageTk.PhotoImage(logo_img)
    
    # Keep a reference to the logo image to prevent garbage collection
    root.logo_photo = logo_photo

    logo_label = tk.Label(root, image=logo_photo, bg='black')
    logo_label.image = logo_photo  # Store a reference to the image
    logo_label.place(x=root.winfo_screenwidth() - 120, y=10)
except Exception as e:
    print(f"Error loading logo image: {e}")

creator_label = tk.Label(root, text="Created by Harsha", fg='white', bg='black', font=('Helvetica', 12))
creator_label.place(x=root.winfo_screenwidth() - 120, y=120)

# Circular start button with gradient
start_button_canvas = Canvas(root, width=200, height=200, bg='black', highlightthickness=0)
start_button_canvas.place(x=root.winfo_screenwidth() // 2 - 100, y=root.winfo_screenheight() // 2 - 100)

def create_circle(x, y, r, canvas_name, **kwargs):  # center coordinates, radius
    return canvas_name.create_oval(x - r, y - r, x + r, y + r, **kwargs)

# circles with gradient
gradient_colors = ['#ff4e50', '#fc913a', '#f9d423']
for i in range(3):
    create_circle(100, 100, 80 - i*20, start_button_canvas, fill=gradient_colors[i], outline='')

# Listening and recognizing animations
listening_label = tk.Label(root, text="Listening...", fg='white', bg='black', font=('Helvetica', 14))
recognizing_label = tk.Label(root, text="Recognizing...", fg='white', bg='black', font=('Helvetica', 14))

# Response text area
response_text = tk.Text(root, height=10, width=80, bg='black', fg='white', font=('Helvetica', 12))
response_text.place(x=root.winfo_screenwidth() // 2 - 600, y=root.winfo_screenheight() - 200)

def show_listening():
    listening_label.place(x=root.winfo_screenwidth() // 2 - 60, y=root.winfo_screenheight() // 2 + 150)
    root.update()

def hide_listening():
    listening_label.place_forget()
    root.update()

def show_recognizing():
    recognizing_label.place(x=root.winfo_screenwidth() // 2 - 70, y=root.winfo_screenheight() // 2 + 150)
    root.update()

def hide_recognizing():
    recognizing_label.place_forget()
    root.update()

def listen():
    """Listen for audio and convert it to text"""
    show_listening()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        hide_listening()
        show_recognizing()
        try:
            command = recognizer.recognize_google(audio)
            print(f"You said: {command}")
            response_text.insert(tk.END, "You: " + command + "\n")
            response_text.see(tk.END)
            return command.lower()
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            response_text.insert(tk.END, "Shakti: Sorry, I did not understand that.\n")
            response_text.see(tk.END)
            return None
        except sr.RequestError:
            print("Sorry, the service is down.")
            response_text.insert(tk.END, "Shakti: Sorry, the service is down.\n")
            response_text.see(tk.END)
            return None
        finally:
            hide_recognizing()

def get_response(prompt):
    """Get response from OpenAI's GPT"""
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content'].strip()

def open_application(command):
    if "chrome" in command.lower():
        speak("Opening Google Chrome")
        subprocess.Popen(["C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"])
        speak("Google Chrome is open.")
    elif "youtube" in command.lower():
        speak("Opening Youtube")
        subprocess.Popen(["C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe", "https://www.youtube.com"])
        speak("Youtube is open.")
    elif "spotify" in command.lower():
        speak("Opening Spotify")
        subprocess.Popen(["C:\Program Files\WindowsApps\SpotifyAB.SpotifyMusic_1.240.599.0_x64__zpdnekdrzrea0\Spotify.exe"])
        speak("Spotify is open.")
    elif "whatsapp" in command.lower():
        speak("Opening Whatsapp")
        webbrowser.open("https://web.whatsapp.com")
        speak("Whatsapp is open.")
    else:
        speak("Application not recognized or not supported.")
        
def speak(text):
    """Convert text to speech"""
    response_text.insert(tk.END, "Shakti: " + text + "\n")
    response_text.see(tk.END)
    engine.say(text)
    engine.runAndWait()

def shakti():
    """Main function to run the assistant"""
    speak("Hello, I'm your voice assistant, Shakti. How can I help you today?")
    while True:
        command = listen()
        if command:
            if "ok bye" in command:
                speak("Bye!")
                break
            elif "open" in command:
                open_application(command)
            elif "what is shakti" in command or "what does this model 'shakti' do" in command:
                predefined_response = (
                    "Shakti is your personal voice assistant designed to help you with various tasks, "
                    "such as opening applications, providing information, and more. Just ask, and I'm here to assist you!"
                )
                speak(predefined_response)
            else:
                response = get_response(command)
                print(f"Shakti: {response}")
                speak(response)

def on_start_button_click(event):
    """Handle start button click event"""
    threading.Thread(target=shakti).start()

# Start button click event
start_button_canvas.bind("<Button-1>", on_start_button_click)

root.mainloop()


<>:142: SyntaxWarning: invalid escape sequence '\P'
<>:142: SyntaxWarning: invalid escape sequence '\P'
C:\Users\harsh\AppData\Local\Temp\ipykernel_14380\2590919451.py:142: SyntaxWarning: invalid escape sequence '\P'
  subprocess.Popen(["C:\Program Files\WindowsApps\SpotifyAB.SpotifyMusic_1.240.599.0_x64__zpdnekdrzrea0\Spotify.exe"])


Listening...
You said: tell me something about yourself
Shakti: I am a language model AI designed to assist with a wide range of tasks such as providing information, answering questions, and engaging in casual conversation. I am constantly learning and improving to better serve users like you. How can I assist you today?
Listening...
You said: open YouTube
Listening...
Listening...
Sorry, the service is down.
Listening...
Sorry, the service is down.Sorry, the service is down.



Exception in thread Thread-5 (shakti):
Traceback (most recent call last):
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\urllib\request.py", line 1344, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1331, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1377, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1326, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1085, in _send_output
    self.send(msg)
  File "C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 1029, in send
 